# Cullinan Pincell Practice
Code from OpenMC examples/modeling a pin-cell

In [ ]:
%matplotlib inline
import openmc

## Define Materials

In [ ]:
uo2 = openmc.Material(1,'uo2')
print(uo2)

In [ ]:
mat = openmc.Material()
print(mat)

In [ ]:
help(uo2.add_nuclide)

In [ ]:
# add nuclides to uo2
uo2.add_nuclide('U235', 0.03)
uo2.add_nuclide('U238', 0.97)
uo2.add_nuclide('O16', 2.0)

In [ ]:
uo2.set_density('g/cm3', 10.0)

In [ ]:
zirconium = openmc.Material(name='zirconium')
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.6)

water = openmc.Material(name='h2o')
water.add_nuclide('H1', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3', 1.0)

In [ ]:
water.add_s_alpha_beta('c_H_in_H2O')

In [ ]:
materials = openmc.Materials([uo2, zirconium, water])

In [ ]:
materials = openmc.Materials()
materials.append(uo2)
materials += [zirconium, water]
isinstance(materials,list)

### Export Materials as XML

In [ ]:
materials.export_to_xml()
!cat materials.xml

In [ ]:
water.remove_nuclide('O16')
water.add_element('O', 1.0)

materials.export_to_xml()
!cat materials.xml

## Cross Sections

In [ ]:
!cat $OPENMC_CROSS_SECTIONS | head -n 10
print('    ...')
!cat $OPENMC_CROSS_SECTIONS | tail -n 10

### Enrichment

In [ ]:
uo2_three = openmc.Material()
uo2_three.add_element('U',1.0, enrichment=3.0)
uo2_three.add_element('O', 2.0)
uo2_three.set_density('g/cc', 10.0)

### Mixtures

In [ ]:
# create PuO2 material
puo2 = openmc.Material()
puo2.add_nuclide('Pu239', 0.94)
puo2.add_nuclide('Pu240', 0.06)
puo2.add_nuclide('O16', 2.0)
puo2.set_density('g/cm3', 11.5)

# create the mixture
mox = openmc.Material.mix_materials([uo2, puo2], [0.97, 0.03], 'wo')

## Define Geometry

In [ ]:
sphere = openmc.Sphere(r=1.0)

In [ ]:
inside_sphere = -sphere
outside_sphere = +sphere

In [ ]:
print((0,0,0) in inside_sphere, (0,0,2) in inside_sphere)
print((0,0,0) in outside_sphere, (0,0,2) in outside_sphere)

In [ ]:
z_plane = openmc.ZPlane(0)
northern_hemi = -sphere & +z_plane

In [ ]:
northern_hemi.bounding_box

In [ ]:
cell = openmc.Cell()
cell.region = northern_hemi

In [ ]:
cell.fill = water

## Create Universe

In [ ]:
universe = openmc.Universe()
universe.add_cell(cell)

In [ ]:
universe.plot(width=(2.0, 2.0))

In [ ]:
universe.plot(width=(2.0, 2.0), basis = 'xz')

In [ ]:
universe.plot(width=(2.0, 2.0), basis='xz', colors={cell: 'fuchsia'})

## Pincell Geometry

In [ ]:
fuel_outer_rad = openmc.ZCylinder(r=0.39)
clad_inner_rad = openmc.ZCylinder(r=0.40)
clad_outer_rad = openmc.ZCylinder(r=0.46)

In [ ]:
fuel_region = -fuel_outer_rad
gap_region = +fuel_outer_rad & -clad_inner_rad
clad_region = +clad_inner_rad & -clad_outer_rad

In [ ]:
fuel = openmc.Cell(name='fuel')
fuel.fill = uo2
fuel.region = fuel_region

gap = openmc.Cell(name='air gap')
gap.region = gap_region

clad = openmc.Cell(name='clad')
clad.fill = zirconium
clad.region = clad_region

In [ ]:
pitch = 1.26
left = openmc.XPlane(-pitch/2, boundary_type='reflective')
right = openmc.XPlane(pitch/2, boundary_type='reflective')
bottom = openmc.YPlane(-pitch/2, boundary_type='reflective')
top = openmc.YPlane(pitch/2, boundary_type='reflective')

In [ ]:
water_region = +left & -right & +bottom & - top & + clad_outer_rad

moderator = openmc.Cell(name='moderator')
moderator.fill = water
moderator.region = water_region

In [ ]:
box = openmc.rectangular_prism(width=pitch, height=pitch, boundary_type='reflective')
type(box)

In [ ]:
water_region = box & +clad_outer_rad

In [ ]:
root_universe = openmc.Universe(cells=(fuel, gap, clad, moderator))

geometry = openmc.Geometry()
geometry.root_universe = root_universe
geometry.export_to_xml()
!cat geometry.xml

## Source definition

In [ ]:
# create a point source
point = openmc.stats.Point((0,0,0))
source = openmc.Source(space=point)


In [ ]:
settings = openmc.Settings()
settings.source = source
settings.batches = 100
settings.inactive = 10
settings.particles = 1000

In [ ]:
settings.export_to_xml()
!cat settings.xml

## tallies


In [ ]:
cell_filter = openmc.CellFilter(fuel)

tally = openmc.Tally(1)
tally.filters = [cell_filter]

In [ ]:
tally.nuclides = ['U235']
tally.scores = ['total','fission','absorption','(n,gamma)']

In [ ]:
tallies = openmc.Tallies([tally])
tallies.export_to_xml()
!cat tallies.xml

In [ ]:
openmc.run()

In [ ]:
!cat tallies.out

## Plot Geometry

In [ ]:
plot = openmc.Plot()
plot.filename = 'pinplot'
plot.width = (pitch,pitch)
plot.pixels = (200,200)
plot.color_by = 'material'
plot.colors = {uo2: 'yellow',water: 'blue'}

In [ ]:
plots = openmc.Plots([plot])
plots.export_to_xml()
!cat plots.xml

In [ ]:
openmc.plot_geometry()

In [ ]:
from IPython.display import Image
Image('pinplot.png')

In [ ]:
plot.to_ipython_image()